# Setting Up All Artifacts details

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
# "/local/mnt/workspace/snpe/2.24.0.240626"
os.environ['SNPE_ROOT']="/local/mnt/workspace/snpe/2.29.0.241129"#set up your snpe path here.
os.environ['RAW_FILE_FOLDER']="raw"
os.environ['DLC32']="models/yolo_nas_fp32.dlc"
os.environ['DLC8']="models/yolo_nas_w8a8.dlc"
os.environ['TARGET_INPUT_LIST']="input.txt"
os.environ['ONDEVICE_FOLDER']="yolonas"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="728b7a92" #change with your device-id. Use command "adb devices" to get devices names.
os.environ['SNPE_TARGET_ARCH']="aarch64-android"
os.environ['SNPE_TARGET_STL']="libc++_shared.so"

In [ ]:
## Note- Use python3.8 or above for generating onnx
!pip install super-gradients==3.1.2
import torch
from super_gradients.training import models
from super_gradients.common.object_names import Models
import cv2
import numpy as np
import os

## Getting The dataset
Please, fill coco dataset link in below code block.

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip -q --show-progress
!unzip val2017.zip
!mkdir "raw"

In [ ]:
files = os.listdir('val2017') #val2017 is the datatset folder path. Keeping only 15 images.
for file in files[15:]:
    os.remove("val2017/"+file)

In [ ]:
%%bash
rm -rf coco
rm -rf coco2017labels.zip
rm -rf val2017.zip

## Getting the ONNX Model

In [ ]:
os.makedirs('models', exist_ok=True)

In [ ]:
model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")
# Prpare model for conversion
# Input size is in format of [Batch x Channels x Width x Height] where 640 is the standard COCO dataset dimensions
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, 320, 320])
# Create dummy_input
dummy_input = torch.randn([1, 3, 320, 320], device="cpu")
# Convert model to onnx
torch.onnx.export(model, dummy_input, "models/yolo_nas_s.onnx", opset_version=11)

#### Getting the FP32 Model

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc -i models/yolo_nas_s.onnx -o app/src/main/assets/yolo_nas_s.dlc --out_node 893 --out_node 885

## Preprocessing

In [ ]:
def preprocess(original_image):
    resized_image = cv2.resize(original_image, (320, 320))
    resized_image = resized_image/255
    return resized_image
##Please download Coco2014 dataset and give the path here
dataset_path = "val2017/"
!mkdir -p rawYoloNAS
filenames=[]
for path in os.listdir(dataset_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))
for filename in filenames:
    original_image = cv2.imread(filename)
    img = preprocess(original_image)
    img = img.astype(np.float32)
    img.tofile("raw/"+filename.split("/")[-1].split(".")[0]+".raw")

In [ ]:
%%bash
find raw -name *.raw > input.txt

## Quantize the DLC

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-quantize --input_dlc app/src/main/assets/yolo_nas_s.dlc --input_list input.txt --output_dlc app/src/main/assets/Quant_intermediate_yoloNas_s_320.dlc

<b>- Based on the device where you will execute the model set --htp_socs to sm8750 or sm8650 or sm8550</b>

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-graph-prepare --input_dlc app/src/main/assets/Quant_intermediate_yoloNas_s_320.dlc --set_output_tensors=893,885 --htp_socs=sm8750 --output_dlc=app/src/main/assets/Quant_yoloNas_s_320.dlc